# FP playground: working with lists

In [ ]:
#require "fp"  (* loads the main FP library I wrote for this course *)

Having superficially covered a range of FP core concepts in the introduction lecture, we will now put this knowledge into practice by learning about list processing. This is a hands-on tutorial with lots of examples for you to cut your teeth on!

Along the way, we will learn about FP concepts such as algebraic types, record types, pattern matching, ...

## 1. What is a list?

In FP, a <span class="emph">list</span> is an ordered collection of elements of the same type. OCaml offers the following syntax for writing lists (note: elements are separated by `;`, not `,`): 

In [ ]:
let x = [ 4; 2; 35; 0 ]

In [ ]:
[ "hello"; "cambridge" ]

In [ ]:
[]

Here, OCaml infers the type of `x` to be `int list`: this is a new kind of type we haven't seen before!

This is an example of a <span class="emph">parameterised type</span>: there can be lists of ints, lists of strings, lists of bools, ... lists of _anything_ really! That is why OCaml uses `'a list` as its native list type. Here, `'a` means “any type” ─ you will see later how we can write functions that operate on `'a list` values (e.g. `length: 'a list -> int`).

Let's look at another example:

In [ ]:
let us_presidents = [ "Obama", 2009, 2017; "Trump", 2017, 2021 ]

OCaml offers the following operators on lists:
- `::` for appending a single element at the front
- `@` for concatenating two lists together.

In [ ]:
x

In [ ]:
let x = 11 :: x

In [ ]:
let empty = [] in
11 :: 4 :: 2 :: 35 :: 0 :: empty

In [ ]:
let x = x @ [ 8 ]

<span class="exo">Exercise 1/A</span>
Use `::` to update `us_presidents` so that it now also includes `"Bush", 2001, 2009`:

In [ ]:
let us_presidents =
  (* your answer here *)

Now, how would add Joe Biden at the end of the list? 🤔

<span class="exo">Impossible exercise 1/B</span>
Use `@` to update `us_presidents` so that it now also includes Joe Biden, whose term started in 2021.

In [ ]:
let us_presidents = (* your answer here *)

## 2. Sum types: representing heterogeneous data

How do we add Joe Biden to the list?
He was elected in 2021, and it is unclear how long he will serve. Thus we simply cannot represent it using a `string * int * int` value... and so we can't add it to a `(string * int * int) list`.

But don't fret! OCaml has a very expressive algebraic type system. We've already learned about product types (e.g. `string * int * int`), so let's now learn about <span class="emph">sum types</span>:

In [ ]:
type maybe_ended =
  | Still_in_office
  | Ended of int

In [ ]:
Still_in_office

In [ ]:
Ended 2005

Here, type `maybe_ended` very naturally expresses the fact that a president may, or may not, have finished his/her term.

We can now pack the last 4 US presidents, including the current one, in a single list:

In [ ]:
let us_presidents =
  [ "Bush", 2001, Ended 2009
  ; "Obama", 2009, Ended 2017
  ; "Trump", 2017, Ended 2021
  ; "Biden", 2021, Still_in_office
  ]

OCaml also has record types, which may be more natural here:

In [ ]:
type president =
  { name : string
  ; started : int
  ; ended : int option
  }

OCaml comes with a pre-defined “option” type, defined as a polymorphic sum type as  follows:
```ocaml
type 'a option = 
  | None
  | Some of 'a
```

In [ ]:
[ "string"]

In [ ]:
[] 

In [ ]:
Some "hello"

In [ ]:
None

We can represent a single president as e.g.

In [ ]:
let bush = { name = "Bush"; started = 2001; ended = Some 2009 }

and access its record fields using the `.` notation: 

In [ ]:
bush.name

So we can now construct our list of US presidents as follows:

In [ ]:
let us_presidents =
  let bush = { name = "Bush"; started = 2001; ended = Some 2009 } in
  let obama = { name = "Obama"; started = 2009; ended = Some 2017 } in
  let trump = { name = "Trump"; started = 2017; ended = Some 2021 } in
  let biden = { name = "Biden"; started = 2021; ended = None } in
  [ bush; obama; trump; biden ]

## 3. The list type as a polymorphic recursive sum type

OCaml allows you to define <span class="emph">recursive types</span>, and this can be used to define a polymorphic list type, like so:

In [ ]:
type 'a my_list =
  | Empty (* empty list *)
  | Append of 'a * 'a my_list
(* an element, potentially followed by more elements *)

In [ ]:
let x = [ 11; 4; 2]

<span class="exo">Exercise 3/A</span> Use the type `my_list` to construct the same list as `x` above, as an `int my_list` value.

In [ ]:
let x = 11 :: 4 :: 2 :: []

In [ ]:
let x_alternative = Append (11, Append (4, Append (2, Empty)))
   

In [ ]:
type 'a binary_tree =  
   | Leaf of 'a
   | Split of 'a binary_tree * 'a binary_tree

In [ ]:
Split (Split (Leaf 50.0000, Leaf 6.), Split (Split (Leaf 5., Leaf 7.), Leaf 25.))

## 4. Pattern matching

OCaml allows you to perform value-name bindings via <span class="emph">pattern-matching</span>.

In [ ]:
let x = "foo", 3.14

In [ ]:
let xs, xf = x

What is going on here? OCaml knows that `x` is a pair, and therefore that it can be identified to a pattern of the form `xs, xf`. When we write
```ocaml
let xs, xf = x
```
OCaml automatically transform this into two let bindings:
- `let xs = [the first element of the pair]`
- `let xf = [the second element of the pair]`

Pattern matching works for pretty much everything 😎, and makes coding _considerably more convenient_. The general keyword for pattern matching is `match`, and is used as follows:

In [ ]:
let y =
  match x with
  | a, b -> a ^ " bar"

In [ ]:
match x with
| a, b -> b +. 2.5

In [ ]:
(* you get an error if you match against the wrong pattern *)
match x with
| a, b, c -> a

The notation is intuitive: we match a given value `x` to a relevant pattern (here, `(a, b)`, and the variable names used inside the pattern (here, `a` and `b`) become bound to their corresponding values in `x` inside the expression that follows the `->` arrow.

Notice that in the code above, we are not actually using `b` anywhere. It is common practice in this case to be explicit about this, and use `_` instead of `b`:

In [ ]:
let y =
  match x with
  | first, _ -> first

<span class="exo">Exercise 4/A</span> Write a function that takes an argument `x : int * int` and returns the sum of the two components.

In [ ]:
let f x = (* your answer here, which should use the match keyword *)

In [ ]:
f (5, 6) (* should be 11 *)

<span class="exo">Exercise 4/B</span> Write a function that takes an argument `x : (int * (int * string))` and returns the string component.

In [ ]:
let f x = (* your answer here *)

In [ ]:
f (3, (14, "well done")) (* should be "well done" *)

In [ ]:
let f (_, (_, a)) = a

Now, let's look at pattern matching for sum types, e.g. to deal with values of type `'a option`:

In [ ]:
let maybe_print x =
  match x with
  | Some msg -> Fp.Utils.print_msg msg
  | None -> ()

In [ ]:
maybe_print (Some "I have a dream")

In [ ]:
maybe_print None

<span class="emph">Pro tip</span> ─ when the first thing you do inside a function is to pattern match the final argument, you can use the `function` keyword directly, like this:

In [ ]:
let maybe_print = function
  | Some msg -> Fp.Utils.print_msg msg
  | None -> ()

<span class="exo">Exercise 4/C</span> Write a function `iter` which takes a function `f : 'a -> unit` and a value `x : 'a option` as arguments, and
- performs `f a` if if `x = Some a`
- does nothing if `x = None`.

In [ ]:
let iter f x = match x with
  | None -> ()
  | Some y -> f y

Now use this `iter` function to rewrite the `maybe_print` function above in a single line. 

In [ ]:
let maybe_print = iter Fp.Utils.print_msg

<span class="exo">Exercise 4/D</span> Write a function `map` which takes a function `f : 'a -> 'b` and a value `x : 'a option` as arguments, and
- returns `Some (f a)` if `x = Some a`
- returns `None` if `x = None`.

In [ ]:
let map f x = (* your answer here *)

In [ ]:
map (fun x -> x + 1) None

In [ ]:
map (fun x -> x + 1) (Some 35)

Congratulations! You have just implemented two of the most important higher-order functions in FP (`iter` and `map`). We'll do the same for lists later in this notebook. But first, let's finish learning about pattern matching.

You can pattern-match records as well, like so:

In [ ]:
let still_in_office = function
  | { ended = None; _ } -> true
  | _ -> false (* _ is a pattern that matches EVERYTHING *)

These two cases shouldn't be switched, because `_` matches everything and would make the second pattern matching case redundant. OCaml complains about this:

In [ ]:
let still_in_office = function
  | _ -> false  (* _ is a pattern that matches EVERYTHING *)
  | { ended = None; _ } -> true

In [ ]:
still_in_office bush

which is equivalent to

In [ ]:
bush.ended

In [ ]:
let still_in_office x =
  match x.ended with
  | None -> true
  | Some _ -> false

Here's another example:

In [ ]:
let check_valid_dates = function
  | { started = s; ended = Some y; _ } -> y >= s
  | _ -> true

In [ ]:
check_valid_dates { name = "test"; started = 2010; ended = Some 2005 }

<span class="exo">Exercise 4/D</span> Write a function `most_recent` that takes two US president records `p1` and `p2`, and returns the most recent one.

In [ ]:
let most_recent p1 p2 = (* your answer here *)

In [ ]:
let most_recent p1 p2 =
  match p1.started > p2.started with
  | true -> p1
  | false -> p2

Finally, lists can be pattern-matched using the `::` constructs we saw above, as follows:

In [ ]:
let first_element = function
  | [] -> None
  | a :: _ -> Some a

In [ ]:
let has_more_than_two_elts x =
  match x with
  | [] -> false
  | [ _ ] -> false
  | [ _; _ ] -> false
  | _ -> true

In [ ]:
let x = [ 3 ] in
has_more_than_two_elts x

I hope you find pattern matching very intuitive!

## 5. List processing

Let's get started with the real stuff! We are now going to write a number of functions applied to lists, to reinforce everything we've learned so far.

<span class="exo">Exercise 5/A</span> Write a function `length : 'a list -> int` that computes the length of a list.

In [ ]:
(* autoformat = Ctrl-l (small L) *)
(* your answer below *)

<span class="exo">Exercise 5/B</span> Write a function `rev : 'a list -> 'a list` that reverse a list

In [ ]:
(* note to self: show the students how to write the TAIL-RECURSIVE version *)

<span class="exo">Exercise 5/C</span> Write a function `nth : 'a list -> int -> 'a option` that extracts the nth element of a list, if it exists (returns None if the list is shorter than `n`).

<span class="exo">Exercise 5/D</span> Write a function `mem : 'a list -> 'a -> bool` such that `mem x a` is `true` if and only if the list `x` contains element `a`.

<span class="exo">Exercise 5/E</span> Write a function `map : ('a -> 'b) -> 'a list -> 'b list` such that `map f [x1; x2; ...; xN]` returns `[f x1; f x2; ...; f xN]`.

<span class="exo">Exercise 5/F</span> Write a function `zip : 'a list -> 'b list -> ('a * 'b) list` such that for two lists `x = [x1; ...; xN]` and `y = [y1; ...; yN]` of the same length, `zip x y` returns `[(x1, y1); ... ; (xN, yN)]`. 

<span class="exo">Exercise 5/G</span> Write a function `map2 : ('a -> 'b -> 'c) -> 'a list -> 'b list -> 'c list` such that `map2 f x y` returns `[f x1 y1; ... ; f xN yN]`.

<span class="exo">Exercise 5/H</span> Rewrite `zip` as an instance of `map2`.

<span class="exo">Exercise 5/I</span> Write a function `init: int -> (int -> 'a) -> 'a list` such that `init n f` equals `[f 1; f2; ...; f n]`.

<span class="exo">Exercise 5/J</span> Write a function `fold: ('b -> 'a -> 'b) -> 'b -> 'a list -> 'b`   such that `fold f init x` equals `f (... (f (f init x1) x2) ...) xN`.

<span class="exo">Exercise 5/K</span> Reuse the `fold` function to write a function `iter : ('a -> unit) -> 'a list -> unit` that performs `f x1; f x2; ... f xN`. Do you see how this can be used to perform a “for loop”?

<span class="exo">Exercise 5/L</span> Reuse the `fold` function to write `filter : 'a list -> ('a -> bool) -> 'a list` such that `filter x f` filters out every element `a` of `x` for which `f a` is false (i.e. only retains those elements `a` for which `f a` is true).

<span class="exo">Exercise 5/M</span> Rewrite `nth` using `fold`.

<span class="exo">Exercise 5/N</span> Rewrite `mem` using `fold`.

From these exercises, it should now be clear that `map`, `fold`, and `map2` (and `fold2` which we haven't had time to implement) are very basic primitives from which many useful functions can be derived. We call these <span class="emph">iterators</emph>.

Time permitting, we will discuss scaling issues typically encountered for recursive functions, and how to avoid these issues by writing so-called <span class="emph">tail recursive</emph> functions.

## 6. Utility functions

Let's write a function that times the execution of a `('a -> 'b) -> 'a -> 'b * float`. 

In [ ]:
let timeit f x =
  let t0 = Unix.gettimeofday () in
  let result = f x in
  let t1 = Unix.gettimeofday () in
  result, t1 -. t0

Infix operators:

This is how OCaml lets you define your own infix operators: for example, in the standard library, ( * ) is defined as:

In [ ]:
let ( * ) x y = Int.mul x y

In [ ]:
6 * 5

In [ ]:
let ( |> ) x f = f x  (* also already defined in the standard library *)

In [ ]:
[ 2; 3; 4 ]
|> map (fun x -> x + 1)
|> map (fun x -> x * x)
|> fold Int.add 0
|> (fun x -> x + 1)
|> fun n -> List.init n (fun x -> x + 4)